# Fine-Tune your Model

잘 동작하는 몇가지 모델을 선정했다면, 이제 finetuning을 할 차례이다.

먼저, 앞 절에서 수행한 내용을 통해 데이터를 준비한다.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

dataset_root = os.path.join(os.getcwd(), "datasets")
housing_path = os.path.join(dataset_root, "housing")

def load_housing_data(housing_path=housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1,
                               test_size=0.2,
                               random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [2]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
housing_cat = housing[["ocean_proximity"]]

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_idx, bedrooms_idx, population_idx, households_idx = 3, 4, 5, 6

class CombinedFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        # Nothing else to do
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_idx]/X[:, households_idx]
        population_per_household = X[:, population_idx]/X[:, households_idx]
        # bedrooms_per_room은 선택
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_idx]/X[:, rooms_idx]
            return np.c_[X,
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X,
                         rooms_per_household,
                         population_per_household]

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('features_adder', CombinedFeaturesAdder()),
    ('std_scaler', StandardScaler())
])

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_features = list(housing_num)
cat_features = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", OneHotEncoder(), cat_features),
])

In [6]:
housing_prepared = full_pipeline.fit_transform(housing)

## Grid Search
좋은 hyperparameter의 조합을 하나하나 직접 찾는 것은 매우 지루하며 시간이 많이 드는 작업이다. 

사이킷런의 `GridSearchCV`는 이를 수월하게 해준다. 실험하고자 하는 hyperparameter와 그 값을 지정해주기만 하면, 모든 가능한 조합들을 cross-validation을 통해 평가할 수 있다.

또한, `GridSearchCV`의 `refit` parameter를 `True`(기본 설정 값)로 하면 가장 좋은 estimator를 찾은 후, 이를 전체 training set(validation 포함)으로 학습시켜준다.

다음 코드는 `param_grid`의 첫번째 dict에서 3x4, 두번째 dict에서 3x2, 총 18가지 조합에서 5-fold cross validation(`cv=5`)를 수행한다. (이전에 만들어 두었던 `add_bedrooms_per_room`와 같은 hyperparameter의 조합도 가능함)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest_reg = RandomForestRegressor()

param_grid = [
    {'n_estimators':[3, 10, 30], 'max_features':[2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features':[2, 3, 4]}
]

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

가장 좋은 성능을 낸 hyperparameter 조합은 다음과 같이 확인할 수 있다.

In [8]:
print(grid_search.best_params_)

{'max_features': 8, 'n_estimators': 30}


위 결과를 보면, 8과 30으로 설정한 hyperparameter에서 최대값이 나왔으므로, 더 큰 값을 통해 학습시키면 성능이 증가될 것이라고 유추할 수 있다.

가장 좋은 성능을 낸 estimator를 직접 확인하는 방법은 다음과 같다.

In [9]:
print(grid_search.best_estimator_)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=8, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


수행한 각 hyperparameter별 evaluation score는 다음과 같이 확인할 수 있다.

In [10]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

64123.16731648048 {'max_features': 2, 'n_estimators': 3}
55701.499298824536 {'max_features': 2, 'n_estimators': 10}
52908.593584518494 {'max_features': 2, 'n_estimators': 30}
60748.307794323 {'max_features': 4, 'n_estimators': 3}
53156.74278987194 {'max_features': 4, 'n_estimators': 10}
50335.364272163184 {'max_features': 4, 'n_estimators': 30}
59275.986714357074 {'max_features': 6, 'n_estimators': 3}
52088.74922050507 {'max_features': 6, 'n_estimators': 10}
50199.49793554348 {'max_features': 6, 'n_estimators': 30}
59298.90878833885 {'max_features': 8, 'n_estimators': 3}
52463.44930223852 {'max_features': 8, 'n_estimators': 10}
50160.65052584281 {'max_features': 8, 'n_estimators': 30}
62924.011201053596 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54527.90119816551 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59865.923981941 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52844.40251717208 {'bootstrap': False, 'max_features': 3, 'n_estimators': 1

## Randomized Search

Randomized Search는 hyperparameter의 무작위 조합으로 탐색하는 방법이다. 이는 hyperparameter의 **search space**가 클 경우(특히, regularization과 같이 연속적인 값의 space인 경우)에 사용하는 것이 좋다.

사이킷런의 `RandomizedSearchCV`를 통해 수행할 수 있다.

## Ensemble Methods

Ensemble은 여러 모델을 조합하는 방법이다. 이는 단일 모델보다 좋은 성능을 내는 경우가 많으며 특히 각 모델이 아주 다른 형태의 error를 출력하는 경우에 좋다.

## Analyze Best Models and Their Errors

좋은 성능의 모델을 분석해보면, 문제에 대한 좋은 insight를 얻을 수 있다. 

예를 들어, 여기서 만든 `RandomForestRegressor`를 통해 정확한 prediction에 대한 각 feature의 상대적인 중요도를 확인할 수 있다.

In [11]:
feature_importances = grid_search.best_estimator_.feature_importances_
print(feature_importances)

[6.89758043e-02 5.95274675e-02 4.39192989e-02 1.66218187e-02
 1.53633739e-02 1.48873346e-02 1.45469111e-02 3.71177599e-01
 4.53109015e-02 1.10934053e-01 6.60581486e-02 8.13089887e-03
 1.58591176e-01 9.50021229e-05 3.15092388e-03 2.70928784e-03]


좀더 확인하기 쉽게 각 feature의 이름과 함께 나타내보자.

In [12]:
extra_features = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_features = list(cat_encoder.categories_[0])

feature_names = num_features + extra_features + cat_one_hot_features
sorted(zip(feature_importances, feature_names), reverse=True)

[(0.371177599371965, 'median_income'),
 (0.15859117613800602, 'INLAND'),
 (0.11093405268663073, 'pop_per_hhold'),
 (0.06897580426161648, 'longitude'),
 (0.06605814861128068, 'bedrooms_per_room'),
 (0.059527467548213, 'latitude'),
 (0.04531090147964578, 'rooms_per_hhold'),
 (0.04391929893274456, 'housing_median_age'),
 (0.016621818728125223, 'total_rooms'),
 (0.015363373859565852, 'total_bedrooms'),
 (0.014887334615080117, 'population'),
 (0.014546911052898074, 'households'),
 (0.008130898868971096, '<1H OCEAN'),
 (0.003150923878604241, 'NEAR BAY'),
 (0.0027092878437575073, 'NEAR OCEAN'),
 (9.500212289567969e-05, 'ISLAND')]

위의 feature별 중요도 정보를 통해 불필요한 feature를 제거할 수도 있다.

## Evaluate Your System on the Test Set

이제 최종 모델을 test set으로 평가해보자. 

test set 데이터를 `full_pipeline`을 통해 변환시킨 후, 최종 모델로 평가한다. 여기서 주의할 점은 `full_pipeline`의  `fit_transform()`이 아닌 `transform()`을 사용해야 한다는 점이다.

In [13]:
from sklearn.metrics import mean_squared_error

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("Final RMSE:", final_rmse)

Final RMSE: 47913.35968957618


만약, estimation value만으로 모델을 확신하기 충분하지 않다면(예를 들면, 현재 운영중인 모델의 성능보다 0.1 % 더 나은 경우) 이 estimation이 얼마나 정확한지 알고 싶을 것이다.

이러한 경우, `scipy.stats.t.interval()`을 통해 **confidence interval(신뢰 구간)**을 계산할 수도 있다.

In [14]:
from scipy import stats
# 여기서는 95%로 설정
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([45914.88142021, 49831.75431041])